In [42]:
import json
import joblib
from tqdm import tqdm
from pathlib import Path
from timeit import default_timer as timer

import pandas as pd
import numpy as np
from io import BytesIO
from PIL import Image
import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
from timm import create_model
from torch.optim.lr_scheduler import OneCycleLR

from accelerate import Accelerator

from isic_helper import DotDict
from isic_helper import get_folds
from isic_helper import compute_auc, compute_pauc
from isic_helper import set_seed
from isic_helper import time_to_str

In [43]:
cfg = DotDict()
cfg.infer = False
cfg.cpu = False
cfg.mixed_precision = None

cfg.image_size = 384
cfg.lr = 5e-4
cfg.num_epochs = 10
cfg.seed = 2022
cfg.train_batch_size = 256
cfg.train_num_worker = 2
cfg.val_batch_size = 256
cfg.val_num_worker = 2
cfg.log_every = 10

cfg.models_output_dir = "models"
cfg.model_name = "resnet18_v1"

In [44]:
INPUT_PATH = Path("../input/isic-2024-challenge/")
MODELS_OUTPUT_PATH = Path(f"{cfg.models_output_dir}")
MODELS_OUTPUT_PATH.mkdir(exist_ok=True)

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False)
train_images = h5py.File(INPUT_PATH / "train-image.hdf5", mode="r")

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
train_metadata = train_metadata.sample(frac=0.05, random_state=cfg.seed)
print(f"Train data size: {train_metadata.shape}")

Train data size: (20053, 57)


In [45]:
id_column = "isic_id"
target_column = "final_target"
group_column = "patient_id"

train_ids = train_metadata[id_column]
groups = train_metadata[group_column]
folds = train_metadata["fold"]
y_train = train_metadata[target_column]

In [46]:
accelerator = Accelerator(cpu=cfg.cpu, mixed_precision=cfg.mixed_precision)

In [47]:
accelerator.device

device(type='cuda')

In [48]:
def dev_augment(image_size):
    transform = A.Compose([
        A.Resize(image_size, image_size),
#         A.Normalize(
#             mean=[0., 0., 0.],
#             std=[1, 1, 1],
#             max_pixel_value=255.0,
#             p=1.0
#         ),
        ToTensorV2()
    ], p=1.)
    return transform

def val_augment(image_size):
    transform = A.Compose([
        A.Resize(image_size, image_size),
#         A.Normalize(
#             mean=[0., 0., 0.],
#             std=[1, 1, 1],
#             max_pixel_value=255.0,
#             p=1.0
#         ),
        ToTensorV2()
    ], p=1.)
    return transform

class ISICDataset(Dataset):
    def __init__(self, metadata, images, augment, infer=False):
        self.metadata = metadata
        self.images = images
        self.augment = augment
        self.length = len(self.metadata)
        self.infer = infer
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        data = self.metadata.iloc[index]
        
        image = np.array(Image.open(BytesIO(self.images[data[id_column]][()])))
        image = self.augment(image=image)["image"]
        
        record = {
            "image": image
        }
        
        if not self.infer:
            target = data[target_column]
            record["target"] = torch.tensor(target).long()
        
        return record

class ISICNet(nn.Module):
    def __init__(self, arch="resnet18", pretrained=False, infer=False):
        super(ISICNet, self).__init__()
        self.infer = infer
        self.model = create_model(model_name=arch, pretrained=pretrained, in_chans=3,  num_classes=0, global_pool='')
        self.classifier = nn.Linear(self.model.num_features, 1)
        
        self.dropouts = nn.ModuleList([nn.Dropout(0.5) for i in range(5)])
        
    def forward(self, batch):
        image = batch["image"]
        image = image.float() / 255
        
        x = self.model(image)
        bs = len(image)
        pool = F.adaptive_avg_pool2d(x, 1).reshape(bs,-1)
        print(pool.size)
        
        if self.training:
            logit = 0
            for i in range(len(self.dropouts)):
                logit += self.classifier(self.dropouts[i](pool))
            logit = logit/len(self.dropouts)
        else:
            logit = self.classifier(pool)
        return logit

In [50]:
best_num_epochs = {}
val_auc_scores = {}
val_pauc_scores = {}
all_folds = np.sort(folds.unique())
oof_predictions = np.zeros(train_metadata.shape[0])
for fold in all_folds:
    set_seed(cfg.seed)
    
    print(f"Running fold: {fold}")
    dev_index = folds != fold
    val_index = folds == fold
    
    dev_metadata = train_metadata[train_metadata["fold"] != fold].reset_index(drop=True);print(f"Dev data size: {dev_metadata.shape}")
    val_metadata = train_metadata[train_metadata["fold"] == fold].reset_index(drop=True);print(f"Val data size: {val_metadata.shape}")
    
    dev_dataset = ISICDataset(dev_metadata, train_images, augment=dev_augment(image_size=cfg.image_size))
    val_dataset = ISICDataset(val_metadata, train_images, augment=val_augment(image_size=cfg.image_size))

    dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=cfg.train_batch_size, num_workers=cfg.train_num_worker)
    val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=cfg.val_batch_size, num_workers=cfg.val_num_worker, drop_last=False)
    
    net = ISICNet(pretrained=True)
    net = net.to(accelerator.device)
        
    optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr / 5)
    lr_scheduler = OneCycleLR(optimizer=optimizer, max_lr=cfg.lr, epochs=cfg.num_epochs, steps_per_epoch=len(dev_dataloader))

    net, optimizer, dev_dataloader, val_dataloader, lr_scheduler = accelerator.prepare(
        net, optimizer, dev_dataloader, val_dataloader, lr_scheduler
    )
    
    print("Ready to train")
    
    overall_step = 0
    starting_epoch = 0
    best_pauc_score = -np.Inf
    best_auc_score = -np.Inf
    best_epoch = None
    best_val_preds = None

    for epoch in range(starting_epoch, cfg.num_epochs):
        net.train()
        for step, batch in tqdm(enumerate(dev_dataloader)):
            # We could avoid this line since we set the accelerator with `device_placement=True`.
            batch = {k: v.to(accelerator.device) for k, v in batch.items()}
            
            optimizer.zero_grad()
            
            outputs = net(batch)
            loss = F.binary_cross_entropy_with_logits(outputs, batch["target"].unsqueeze(1))
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()

        net.eval()
        val_preds = []
        val_y = []
        for step, batch in tqdm(enumerate(val_dataloader)):
            # We could avoid this line since we set the accelerator with `device_placement=True`.
            batch = {k: v.to(accelerator.device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = net(batch)
            val_preds_batch = torch.sigmoid(outputs)
            val_y_batch = batch["target"]
            val_preds_batch, val_y_batch = accelerator.gather_for_metrics((val_preds_batch, val_y_batch))
            val_preds.append(val_preds_batch.data.cpu().numpy().reshape(-1))
            val_y.append(val_y_batch.data.cpu().numpy().reshape(-1))
            
        val_preds = np.concatenate(val_preds)
        val_y = np.concatenate(val_y)
        auc = compute_auc(val_y, val_preds) 
        pauc = compute_pauc(val_y, val_preds, min_tpr=0.80)
        
        if pauc >= best_pauc_score:
            best_auc_score = auc
            best_pauc_score = pauc
            best_epoch = epoch
            best_val_preds = val_preds
        print(f"Epoch pauc: {pauc} | Best auc: {best_auc_score} | Best pauc: {best_pauc_score} | Best epoch: {best_epoch}")
        
        output_dir = f"fold_{fold}/model_{cfg.model_name}_epoch_{epoch}"
        if cfg.models_output_dir is not None:
            output_dir = Path(f"{cfg.models_output_dir}/{output_dir}")
        accelerator.save_state(output_dir)
    
    best_num_epochs[f"fold_{fold}"] = best_epoch
    val_auc_scores[f"fold_{fold}"] = best_auc_score
    val_pauc_scores[f"fold_{fold}"] = best_pauc_score
    
    oof_predictions[val_index] = best_val_preds
    print("\n")
    break

Running fold: 1
Dev data size: (16040, 57)
Val data size: (4013, 57)
Ready to train


0it [00:03, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 41.12 MiB is free. Process 2724 has 15.85 GiB memory in use. Of the allocated memory 15.48 GiB is allocated by PyTorch, and 81.53 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [27]:
a = next(iter(dev_dataloader))

In [29]:
a

{'index': tensor([ 2350, 15414, 13373, 10416,  4991,  2901, 12102, 15483, 11463, 10048,
          9345, 14708,  9903,  7172,  9658, 13044,  2521,  1352,  4372,  1940,
         15880,  7643, 13040,  7385, 10210, 14400,  9433, 12534,  3589,  5844,
          2507,  3714, 12036,  3258, 11688,  6212,  7210,  3391, 12508,  1136,
         14370,   107,  9628,  4073,  6349, 10216, 13689, 11017,  3251, 12248,
          1832,  6901, 12937,  9251,  4381,  1875,  6479,  3925,  1049,  8917,
          7275,  7153,  3854, 15845, 15950,  9324,  6762,  2575,  8761,  6833,
         15619, 13328,  7501,   883,  9683, 11186,  5151,   432,  2699,  4012,
         15583,  2856,  4686, 14620,  5874, 12601, 12726, 10159,  3747, 11088,
          6408,   670, 10228,  7010, 13306,  6325,  1021,  2605,  7330, 10949,
         15685,  4031,  6529,  5371, 12842,  6658, 15819,  8607, 10946, 15179,
         10567, 14991,  8351, 15246,  7604,   720,  6539, 15244, 11089, 12561,
           540,  2947,  5796,  5620, 11055,

In [ ]:
oof_preds_df = pd.DataFrame({
    id_column: train_ids,
    group_column: groups,
    "fold": folds,
    target_column: y_train,
    f"oof_{cfg.model_name}": oof_predictions
})
oof_preds_df.to_csv(f"oof_preds_{cfg.model_name}.csv")
oof_preds_df.head()

In [ ]:
best_num_epochs

In [ ]:
val_auc_scores

In [ ]:
val_pauc_scores

In [ ]:
cv_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{cfg.model_name}"])
cv_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{cfg.model_name}"], min_tpr=0.8)

cv_auc_avg = np.mean(list(val_auc_scores.values()))
cv_pauc_avg = np.mean(list(val_pauc_scores.values()))

In [ ]:
print(f"CV AUC OOF: {cv_auc_oof}")
print(f"CV PAUC OOF: {cv_pauc_oof}")
print(f"CV AUC AVG: {cv_auc_avg}")
print(f"CV PAUC AVG: {cv_pauc_avg}")

In [ ]:
params = vars(cfg)
params = {k: v for k, v in params.items() if not k.startswith("_")}

metadata = {
    "params": params,
    "best_num_epochs": best_num_epochs,
    "val_auc_scores": val_auc_scores,
    "val_pauc_scores": val_pauc_scores,
    "cv_auc_oof": cv_auc_oof,
    "cv_pauc_oof": cv_pauc_oof,
    "cv_auc_avg": cv_auc_avg,
    "cv_pauc_avg": cv_pauc_avg
}

with open("run_metadata.json", "w") as f:
    json.dump(metadata, f)